In [0]:
# Check that we have GPU access
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

# Install dependencies and pre-reqs
!pip install pydrive

In [2]:
# Download all the images

import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

list_string = "'%s' in parents and trashed=false"

# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

for folder in drive.ListFile({'q':list_string % 'root'}).GetList():
  if folder['title'] == 'tf_training':
    root = os.getcwd()
    for f in drive.ListFile({'q':list_string % folder['id']}).GetList():
      os.chdir(root)
      if f['title'] == 'annotations.pbtext':
        f.GetContentFile(f['title'])
      if f['title'] == 'ssd_mobilenet_v1_coco.config':
        f.GetContentFile(f['title'])
      if f['title'] == 'images':
        !mkdir images
        os.chdir('images')
        images_dir = os.getcwd()
        for images in drive.ListFile({'q':list_string % f['id']}).GetList():
          os.mkdir(images['title'])
          os.chdir(images['title'])
          for image in drive.ListFile({'q':list_string % images['id']}).GetList():
            image.GetContentFile(image['title'])
          print("Finished copying '%s' images into %s", images['title'], os.getcwd())
          os.chdir(images_dir)
        continue
      if f['title'] == 'annotations':
        !mkdir -p /content/annotations
        os.chdir('annotations')
        for xml in drive.ListFile({'q':list_string % f['id']}).GetList():
          xml.GetContentFile(xml['title'])
        print("Finished copying annotations into %s", os.getcwd())
        continue
os.chdir(root)

tf_cloud_training.ipynb
annotations
images


In [0]:
# Download tensorflow Models 
!apt-get -y install protoc-compiler
!git clone https://github.com/tensorflow/models
!export PYTHONPATH=$PYTHONPATH:/content/models:/content/models/research:/content/models/research/slim:/content/models/research/object_detection
!cd models/research && protoc --python_out=. object_detection/protos/*.proto && cd /content

## Annotate all the images
!wget https://raw.githubusercontent.com/uvic-auvic/TensorFlowScripts/master/training_locally/generate_tf_record.py
!python generate_tf_record.py


In [0]:
# Start training !
import os
os.chdir('/content')
# If at you are satisfied and want to stop training then just stop this particular process
!python /models/research/object_detection/train.py --train_dir=training --pipeline_config=ssd_mobilenet_v1_coco.config

In [0]:
# Export inference graph and upload to your Drive
!cpkt=$(ls -t training/*.index | awk -F '.index' 'NR==1 {print $1}')
!python /models/research/object_detection/export_inference_graph.py \
  --input_type image_tensor \
  --pipeline_config_path ssd_mobilenet_v1_coco.config \
  --output_directory inference_graph

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# 1. Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

for f in drive.ListFile({'q':"'root' in parents and trashed=false"}).GetList():
  if f['title'] == 'tf_training':
    file_in_drive = {'parents' : [{u'id': f['id']}]}
    saved_graph = drive.CreateFile(file_in_drive)
    saved_graph.SetContentFile('inference_graph/frozen_inference_graph.pb')
    saved_graph.Upload()
    print('Uploaded Model to Google Drive')

Uploaded Model to Google Drive
